Segmenting And Clustering Neighborhoods in Toronto

Download the wiki section, web scraping to be done
Use BeautifulSoup for web scraping

In [1]:
from bs4 import BeautifulSoup
!wget -q -O 'postalwiki_page.html' 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
print('data downloaded')    

data downloaded


In [2]:
with open('postalwiki_page.html') as file1:
    soup = BeautifulSoup(file1, 'lxml')

In [3]:
print(soup.title.text)

List of postal codes of Canada: M - Wikipedia


In [4]:
t1 =soup.find('table', class_='wikitable sortable')
print(t1.th.text)

Postcode


Point 3 
Fetch the appropriate sections required to create dataframe using the BeautifulSoup object
Create Dataframe with columns PostalCode, Borough, Neighborhood

In [5]:
dict1 = {}
id = 0
heading = []
for tcell in soup.find_all('table', class_='wikitable sortable'):
    for t2 in tcell.find_all('tr'):
        #print(t2.text)
        id = id + 1
        l1 = []
        
        for h1 in t2.find_all('th'):
            #print(h1.text)
            heading.append(h1.text.replace('\n', ''))
            
        for t3 in t2.find_all('td'):
            l1.append(t3.text.replace('\n', ''))
        #print(l1)
        if id != 1 :
            dict1[id] = l1

#print(dict1)

import pandas as pd
d1 = pd.DataFrame.from_dict(dict1, orient='index', columns=heading)

print(d1.columns)

#identify rows that contain 'Not assigned' in 'Borough' column
indexnames = d1[d1['Borough'] == 'Not assigned'].index
print(type(indexnames))

#remove rows that contain 'Borough' as 'Not assigned'
d1.drop(indexnames, axis=0, inplace=True) 
 
print(d1.shape)
d1.head()

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')
<class 'pandas.core.indexes.numeric.Int64Index'>
(210, 3)


,Postcode,Borough,Neighbourhood
4,M3A,North York,Parkwoods
5,M4A,North York,Victoria Village
6,M5A,Downtown Toronto,Harbourfront
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor


'Not Assigned' Neighborhood cells will be same as Borough

In [6]:
d1['Neighbourhood'].str.contains('Not assigned').sum()
d1['Neighbourhood'].replace('Not assigned', d1['Borough'], inplace=True)
d1['Neighbourhood'].str.contains('Not assigned').sum()
d1.head()

,Postcode,Borough,Neighbourhood
4,M3A,North York,Parkwoods
5,M4A,North York,Victoria Village
6,M5A,Downtown Toronto,Harbourfront
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor


Combine multiple 'Neighborhood' that belongs to same postal code

In [7]:
new_df = d1[d1.duplicated(['Postcode'], keep=False)]
t1 = new_df['Postcode'].unique()

for p1 in t1:
    new_df2 = new_df[new_df['Postcode'] == p1]
    #print(new_df2.index)
    str1 = ""
    itr = 0
    key = 0
    for id in new_df2.index:
        if len(str1) == 0:
            str1 = new_df2.loc[id]['Neighbourhood']
        else:
            str1 = str1 + "," + new_df2.loc[id]['Neighbourhood']
        if itr == 0:
            key = id
        else:
            d1.drop(id, axis=0, inplace=True)
        itr = itr + 1
        
    #print(str1)
    #print(key)
    #print(d1.loc[key])
    d1.loc[key]['Neighbourhood'] = str1
    #print(d1.loc[key])
    #print(d1.loc[8])
    #break   
    
print(d1)
print(d1.shape)

    Postcode           Borough  \
4        M3A        North York   
5        M4A        North York   
6        M5A  Downtown Toronto   
7        M6A        North York   
9        M7A  Downtown Toronto   
11       M9A      Queen's Park   
12       M1B       Scarborough   
15       M3B        North York   
16       M4B         East York   
18       M5B  Downtown Toronto   
20       M6B        North York   
23       M9B         Etobicoke   
28       M1C       Scarborough   
32       M3C        North York   
34       M4C         East York   
35       M5C  Downtown Toronto   
36       M6C              York   
39       M9C         Etobicoke   
43       M1E       Scarborough   
48       M4E      East Toronto   
49       M5E  Downtown Toronto   
50       M6E              York   
54       M1G       Scarborough   
57       M4G         East York   
58       M5G  Downtown Toronto   
59       M6G  Downtown Toronto   
63       M1H       Scarborough   
64       M2H        North York   
65       M3H  

Point 5
Create a dataframe containing the geographical coordinates of the postal codes

In [8]:
print ("Download the geospatial data csv file")
!wget -q -O 'geo_data.csv' 'http://cocl.us/Geospatial_data'
print ("data downloaded")

Download the geospatial data csv file
data downloaded


In [9]:
import pandas as pd
geo_data_df = pd.read_csv('geo_data.csv')
geo_data_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
geo_data_df[geo_data_df['Postal Code'] == "M3A"]["Latitude"]
lat1 = []
long1 = []
for val1 in d1['Postcode']:
    #print(val1)
    val2 = (geo_data_df[geo_data_df['Postal Code'] == val1]["Latitude"]).values
    #print(type(val2))
    #print(val2[0])
    lat1.append(val2[0])
    val3 = (geo_data_df[geo_data_df['Postal Code'] == val1]["Longitude"]).values
    long1.append(val3[0])
    
#print(lat1)
d1['Latitude'] = lat1
d1['Longitude'] = long1

d1.head()



,Postcode,Borough,Neighbourhood,Latitude,Longitude
4,M3A,North York,Parkwoods,43.753259,-79.329656
5,M4A,North York,Victoria Village,43.725882,-79.315572
6,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
7,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
9,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494


Explore and Cluster Neighborhoods in Toronto

In [11]:
#select only Borough that contains Toronto
toronto_df = d1[d1['Borough'].str.contains('Toronto')]

Use FourSquare requests to retrieve the venues in neighborhoods of Toronto

In [12]:
# The code was removed by Watson Studio for sharing.

In [13]:

import requests

def getnearbyvenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    
Venues = getnearbyvenues(toronto_df['Neighbourhood'], toronto_df['Latitude'], toronto_df['Longitude'])
Venues.head()


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Harbourfront,43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,Harbourfront,43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Harbourfront,43.65426,-79.360636,Cooper Koo Family YMCA,43.653191,-79.357947,Gym / Fitness Center
3,Harbourfront,43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,Harbourfront,43.65426,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant


In [14]:
Venues.shape
#Venues.groupby('Neighborhood').count()

(351, 7)

Categorize the venues of neighborhoods in Toronto

In [15]:
Venues_OneHot = pd.get_dummies(Venues[['Venue Category']], prefix="", prefix_sep="")
#Venues_OneHot.head()
Venues_OneHot['Neighbourhood'] = Venues['Neighbourhood']
Venues_OneHot.loc[0:10, ('Neighbourhood', 'Bakery')]
Venues_OneHot.shape

(351, 122)

In [16]:
#grouping neighborhood
Venues_grouped = Venues_OneHot.groupby('Neighbourhood').mean().reset_index()
Venues_grouped

,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Sushi Restaurant,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.1,0.0,0.0,0.0,0.000000,0.1,0.0,0.0
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
5,"Cabbagetown,St. James Town",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
6,Central Bay Street,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.10,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
7,"Chinatown,Grange Park,Kensington Market",0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0
8,Christie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
9,Church and Wellesley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00,0.00,0.0,0.0,0.0,0.1,0.000000,0.0,0.0,0.0


In [17]:
#clustering
from sklearn.cluster import KMeans

Venues_cluster = Venues_grouped.drop('Neighbourhood', 1)
print(Venues_cluster.shape)
k1 =KMeans(n_clusters=5)
k1.fit(Venues_cluster)
#k1.labels_[0:10]
print(type(k1.labels_.tolist()))

Venues_grouped['Cluster'] = k1.labels_.tolist()
Venues_grouped.head()

(39, 121)
<class 'list'>


,Neighbourhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,American Restaurant,Arts & Crafts Store,Asian Restaurant,Auto Workshop,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
0,"Adelaide,King,Richmond",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0
1,Berczy Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.0,0.1,0.0,0.0,0
2,"Brockton,Exhibition Place,Parkdale Village",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.1,0.1,0.1,0.2,0.1,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [18]:
toronto_df.head()
#toronto_df.shape

,Postcode,Borough,Neighbourhood,Latitude,Longitude
6,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
9,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494
18,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
35,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
48,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [19]:
#Join 2 data frames that contains neighborhood's geocoordinates and the cluster it belongs to
print(type(Venues_grouped))
print(type(toronto_df))
toronto_venue_cat = toronto_df.join(Venues_grouped.set_index('Neighbourhood'), on='Neighbourhood')
print(type(toronto_venue_cat))
toronto_venue_cat.head()

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
6,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3
9,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
18,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.1,0.0,0.000000,0.0,0.0,0.0,0
35,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,3
48,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0


In [21]:
#import map related libs
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

Create a map that displays the Toronto's clustered neighborhood.

In [22]:
#get toronto's lat and log
m1 = folium.Map(location=[43.65, -79.38], zoom_start=11)

# set color scheme for the clusters
kclusters = 5
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, long, pos, cluster in zip(toronto_venue_cat['Latitude'], toronto_venue_cat['Longitude'], toronto_venue_cat['Neighbourhood'], toronto_venue_cat['Cluster']):
    label = folium.Popup(str(pos) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(m1)
    
m1

Check the info on each cluster

In [23]:
toronto_venue_cat[toronto_venue_cat['Cluster'] == 0]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
9,M7A,Downtown Toronto,Queen's Park,43.662301,-79.389494,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
18,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937,0.0,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.1,0.0,0.000000,0.0,0.0,0.0,0
48,M4E,East Toronto,The Beaches,43.676357,-79.293031,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0
49,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0.0,0.0,0.0,0.0,0.0,...,0.0,0.1,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0
58,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
69,M5H,Downtown Toronto,"Adelaide,King,Richmond",43.650571,-79.384568,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.1,0.0,0.0,0
84,M5J,Downtown Toronto,"Harbourfront East,Toronto Islands,Union Station",43.640816,-79.381752,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0
87,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.1,0.0,0
98,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.1,0
114,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0


In [24]:
toronto_venue_cat[toronto_venue_cat['Cluster'] == 1]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
197,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.38316,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [25]:
toronto_venue_cat[toronto_venue_cat['Cluster'] == 2]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
146,M5N,Central Toronto,Roselawn,43.711695,-79.416936,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2


In [26]:
toronto_venue_cat[toronto_venue_cat['Cluster'] == 3]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
6,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
35,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
59,M6G,Downtown Toronto,Christie,43.669542,-79.422564,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
72,M6H,West Toronto,"Dovercourt Village,Dufferin",43.669005,-79.442259,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
100,M5K,Downtown Toronto,"Design Exchange,Toronto Dominion Centre",43.647177,-79.381576,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
102,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",43.636847,-79.428191,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
116,M5L,Downtown Toronto,"Commerce Court,Victoria Hotel",43.648198,-79.379817,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3
170,M5R,Central Toronto,"The Annex,North Midtown,Yorkville",43.672710,-79.405678,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,0.0,3
199,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",43.653206,-79.400049,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.0,3
212,M4V,Central Toronto,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",43.686412,-79.400049,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3


In [27]:
toronto_venue_cat[toronto_venue_cat['Cluster'] == 4]

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Terminal,...,Swim School,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Vegetarian / Vegan Restaurant,Wine Bar,Yoga Studio,Cluster
145,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0.0,0.0,0.0,0.0,0.0,...,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,4
240,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0.0,0.0,0.0,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,4
